<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Functions" data-toc-modified-id="Functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Functions</a></span></li><li><span><a href="#Import-and-Analysis" data-toc-modified-id="Import-and-Analysis-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import and Analysis</a></span><ul class="toc-item"><li><span><a href="#Dataset-description:" data-toc-modified-id="Dataset-description:-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Dataset description:</a></span></li></ul></li><li><span><a href="#Data-cleaning" data-toc-modified-id="Data-cleaning-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data cleaning</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Sosa:" data-toc-modified-id="Sosa:-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Sosa:</a></span></li></ul></li></ul></li></ul></div>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression


ModuleNotFoundError: No module named 'matplotlib'

# Functions

In [4]:
## Function to explain NA values in a column:

def NA_values(series):
    '''
    Function that takes a Pandas Series and returns a print statement explaining NAs and % of the column
    '''
    
    return print(f'Column name: {series.name}\nTotal values: {int(series.count())}\nNA values: {series.isna().sum()}\n% of NA values: {round(series.isna().mean() * 100,2)}%')


# Import and Analysis

In [5]:
data = pd.read_csv('data/online_shoppers_intention_DATAPTDIC19.csv')

In [6]:
data.head()

,Unnamed: 0,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0.0,0.0,0.0,1.0,0.000000,0.20,0.20,0.0,0.0,Feb,1,1.0,1,1.0,Returning_Visitor,0.0,False
1,1,0.0,0.0,0.0,0.0,2.0,64.000000,0.00,0.10,0.0,0.0,Feb,2,2.0,1,2.0,Returning_Visitor,0.0,False
2,2,0.0,-1.0,0.0,-1.0,1.0,-1.000000,0.20,0.20,0.0,0.0,Feb,4,1.0,9,3.0,Returning_Visitor,0.0,False
3,3,0.0,0.0,0.0,0.0,2.0,2.666667,0.05,0.14,0.0,0.0,Feb,3,2.0,2,4.0,Returning_Visitor,0.0,False
4,4,0.0,0.0,0.0,0.0,10.0,627.500000,0.02,0.05,0.0,0.0,Feb,3,3.0,1,4.0,Returning_Visitor,1.0,False


In [7]:
data.shape

(12330, 19)

In [8]:
data.describe()

,Unnamed: 0,Administrative,Informational,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,SpecialDay,Browser,TrafficType
count,12330.000000,12296.000000,12316.000000,12196.000000,12316.000000,12216.000000,12316.000000,12330.000000,12230.000000,12210.000000
mean,6164.500000,10.428351,0.503979,31.739259,1196.037057,0.027052,0.043003,0.061427,2.350613,4.062899
std,3559.508744,89.580739,1.270701,44.468360,1914.372511,0.129758,0.048527,0.198917,1.720863,4.016304
min,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,-1.000000,1.000000
25%,3082.250000,0.000000,0.000000,7.000000,185.000000,0.000000,0.014286,0.000000,2.000000,2.000000
50%,6164.500000,1.000000,0.000000,18.000000,599.766190,0.003175,0.025124,0.000000,2.000000,2.000000
75%,9246.750000,4.000000,0.000000,38.000000,1466.479902,0.017027,0.050000,0.000000,2.000000,4.000000
max,12329.000000,999.000000,24.000000,705.000000,63973.522230,3.000000,0.200000,1.000000,13.000000,20.000000


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 19 columns):
Unnamed: 0                 12330 non-null int64
Administrative             12296 non-null float64
Administrative_Duration    12316 non-null object
Informational              12316 non-null float64
Informational_Duration     12316 non-null object
ProductRelated             12196 non-null float64
ProductRelated_Duration    12316 non-null float64
BounceRates                12216 non-null float64
ExitRates                  12316 non-null float64
PageValues                 12310 non-null object
SpecialDay                 12330 non-null float64
Month                      12330 non-null object
OperatingSystems           12330 non-null object
Browser                    12230 non-null float64
Region                     12230 non-null object
TrafficType                12210 non-null float64
VisitorType                12310 non-null object
Weekend                    12310 non-null object
R

## Dataset description:
 - Administrative:`float`. Administrative Value. `yet to identify`.
 - Administrative_Duration: `object`. Duration in Administrative Page. `Identify values and change dtype accordingly.`. 
 - Informational: `float`. Informational Value. `yet to identify` 
 - Informational_Duration: `object`. Duration in Informational Page. `Identify values and change dtype accordingly.`
 - ProductRelated: `float.` Product Related Value. `yet to identify` 
 - ProductRelated_Duration: `object`. Duration in Product Related Page. `Identify values and change dtype accordingly.` 
 - BounceRates: `float`. Bounce Rates of a web page. Percentages. The value of "Bounce Rate" feature for a web page refers to the percentage of visitors who enter the site from that page and then leave ("bounce") without triggering any other requests to the analytics server during that session. 
 - ExitRates: `float`. Exit rate of a web page. The value of "Exit Rate" feature for a specific web page is calculated as for all pageviews to the page, the percentage that were the last in the session. 
 - PageValues: `object`. Page values of each web page. The "Page Value" feature represents the average value for a web page that a user visited before completing an e-commerce transaction. `Identify values and change dtype accordingly.` 
 - SpecialDay: Special days like valentine etc. `float`. Closeness to a special date. `dtype correct`. For example, for Valentina’s day, this value takes a nonzero value between February 2 and February 12, zero before and after this date unless it is close to another special day, and its maximum value of 1 on February 8. 
 - Month: Month of the year. `object`. String to identify the month of the year. `clean`. 
 - OperatingSystems: Operating system used. `float`. `Try to explain the values`. 
 - Browser: Browser used. `integer` 
 - Region: Region of the user. `integer`. 
 - TrafficType: Traffic Type. `integer`.  
 - VisitorType: Types of Visitor `object` 
 - Weekend: Weekend or not `bool`  
 - Revenue: Revenue will be generated or not `object`. `Should be bool` 
     

# Data cleaning

### Sosa:

    - Administrative:
        Due to the low number of missing values (0.12%), we'll fill them with the median of the values since its 
        impact won't be noticeable. We can't use mean since it's value represents a category and should assign to 
        an existing value. We have no other issues on this column.
        
    - ProductRelated_Duration
        First off, since we have floats we convert the column dtype to float. This is due a modification on the original
        In this column, we have negative values. The time of a person staying in a webpage cannot be negative, so
        we assume there's a error on the lecture. We'll be replacing them with '0' value.
        The lectures we're given 
    - Month
    - VisitorType
    
Missing values
Outliers
Errors
Transformation

In [10]:
## Administrative column

# before processing: NA_values(data.Administrative)

data.Administrative = data.Administrative.fillna(data.Administrative.median())

In [11]:
## ProductRelated_Duration column

#NA values:

NA_values(data.ProductRelated_Duration)

# Fixing the negative values:
data[data['ProductRelated_Duration'] == '-1'] = '0'

data['ProductRelated_Duration'].value_counts()

Column name: ProductRelated_Duration
Total values: 12316
NA values: 14
% of NA values: 0.11%


/usr/local/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


 0.000000       719
-1.000000        33
 17.000000       21
 11.000000       17
 8.000000        17
               ... 
 553.166667       1
 1523.578030      1
 1045.102381      1
 4622.000000      1
 1314.083333      1
Name: ProductRelated_Duration, Length: 9552, dtype: int64

In [12]:
## Month column

NA_values(data.Month)

Column name: Month
Total values: 12330
NA values: 0
% of NA values: 0.0%


In [13]:
## VisitorType column

NA_values(data.VisitorType)

Column name: VisitorType
Total values: 12310
NA values: 20
% of NA values: 0.16%


### Isaac:

    - Informational_Duration: There is a 'None' value so for this reason the type is an object.. We should change it to 0 and convert all the Series to float. In terms of missing values we got it covered (0.11%). We will fill them with the mean. 0.0% of outliers.
    - PageValues. Same as Informationa_Duration, we got an object called (#Nan). We should also change it and convert all the Series to float. Same for the missing values. 0.0% of outliers.
    - Region. There are only 9 regions. There are some with 'nan' or 'Nan'. My proposal is to remove whole cell as it will end up with bad results.

In [21]:
# Informational_Duration column

data[data['Informational_Duration'] == 'None'] = 0
data["Informational_Duration"] = pd.to_numeric(data["Informational_Duration"])

NA_values(data["Informational_Duration"])
iqr(data, data["Informational_Duration"])

Column name: Informational_Duration
Total values: 12316
NA values: 14
% of NA values: 0.11%
Column: Informational_Duration
Lower outliers: all values lower than nan
Upper outliers: all values higher than nan
Total number of rows with outliers: 0
% of outliers: 0.0%


In [22]:
# PageValues column

data[data['PageValues'] == '#Nan'] = 0
data["PageValues"] = pd.to_numeric(data["PageValues"])

NA_values(data["PageValues"])
iqr(data, data["PageValues"])

Column name: PageValues
Total values: 12310
NA values: 20
% of NA values: 0.16%
Column: PageValues
Lower outliers: all values lower than nan
Upper outliers: all values higher than nan
Total number of rows with outliers: 0
% of outliers: 0.0%


In [25]:
# Region column

#data[data['Region'] == 'nan']
#data[data['Region'] == 'Nan']

#data["Region"] = pd.to_numeric(data["Region"])

ValueError: Unable to parse string "Nan" at position 253